In [6]:
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM
import datasets
from datasets import load_dataset

import sys
import src.evals.data as data_module

import torch

import random

In [2]:
tokenizer = "answerdotai/ModernBERT-large"

In [3]:
chat = load_dataset("sarahpann/rwb_chat")

In [4]:
chat = data_module.create_preference_to_flan_style_dataset(
    task = "sarahpann/rwb_chat",
    split = "train",
    tokenizer_name = tokenizer,
    max_seq_length = 3600,
    prefix = "Which response is the most helpful, relevant, and correct? ",

    dataset_name="sarahpann/rwb_chat",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_chat": ('chosen', 'rejected', 'og_dataset')}
)

chat_hard = data_module.create_preference_to_flan_style_dataset(
    task = "sarahpann/rwb_chat_hard",
    split = "train",
    tokenizer_name = tokenizer,
    max_seq_length = 3600,
    prefix = "Which response is the most helpful, relevant, and correct? ",

    dataset_name="sarahpann/rwb_chat_hard",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_chat_hard": ('chosen', 'rejected', 'og_dataset')}
)

reasoning = data_module.create_preference_to_flan_style_dataset(
    task="sarahpann/rwb_reasoning",
    split='train',
    tokenizer_name=tokenizer,
    max_seq_length=3600,
    prefix="Determine which response is the best choice based on mathematical or programming accuracy. ",

    dataset_name="sarahpann/rwb_reasoning",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_reasoning": ('chosen', 'rejected', 'og_dataset')}
)

safety = data_module.create_preference_to_flan_style_dataset(
    task="sarahpann/rwb_safety",
    split='train',
    tokenizer_name=tokenizer,
    max_seq_length=3600,
    prefix="Which response is the most helpful, relevant, and correct? ",

    dataset_name="sarahpann/rwb_safety",
    dataset_subset="",
    task_column_names={"sarahpann/rwb_safety": ('chosen', 'rejected', 'og_dataset')}
)

Map: 100%|██████████| 740/740 [00:00<00:00, 4697.88 examples/s]


In [5]:
model = AutoModelForMaskedLM.from_pretrained("sarahpann/all_at_once_model")

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [50]:
chat.set_format(type='torch', columns=['input_ids', 'attention_mask'])
chat_hard.set_format(type='torch', columns=['input_ids', 'attention_mask'])
reasoning.set_format(type='torch', columns=['input_ids', 'attention_mask'])
safety.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [51]:
chat_loader = torch.utils.data.DataLoader(chat, batch_size=1, shuffle=False)
chat_hard_loader = torch.utils.data.DataLoader(chat_hard, batch_size=1, shuffle=False)
reasoning_loader = torch.utils.data.DataLoader(reasoning, batch_size=1, shuffle=False)
safety_loader = torch.utils.data.DataLoader(safety, batch_size=1, shuffle=False)

In [18]:
real_tokenizer = AutoTokenizer.from_pretrained(tokenizer)

In [65]:
def eval_dataset(model, subject_dataloader):
    total = 0
    correct = 0

    model.to("cuda")
    model.eval()

    with torch.no_grad():
        for example in subject_dataloader:
            input_ids = example['input_ids'].clone()
            input_ids[:, -2] = real_tokenizer.mask_token_id
            input_ids = input_ids.to("cuda")
            attention_mask = example['attention_mask'].to("cuda")

            output = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = output.logits
            pred = torch.argmax(logits, dim=-1)

            if pred[0, -2] == example['input_ids'][0, -2]:
                correct += 1
            total += 1

            
    print(f"Accuracy: {correct / total}")
    return correct / total

In [66]:
chat_acc = eval_dataset(model, chat_loader)
chat_hard_acc = eval_dataset(model, chat_hard_loader)
reasoning_acc = eval_dataset(model, reasoning_loader)
safety_acc = eval_dataset(model, safety_loader)

Accuracy: 0.7946141479099679
Accuracy: 0.2521551724137931
Accuracy: 0.76659678546471
Accuracy: 0.7918918918918919


In [67]:
final_results = {
    "chat": chat_acc,
    "chat_hard": chat_hard_acc,
    "reasoning": reasoning_acc,
    "safety": safety_acc
}

print(final_results)

{'chat': 0.7946141479099679, 'chat_hard': 0.2521551724137931, 'reasoning': 0.76659678546471, 'safety': 0.7918918918918919}
